# Final Segmentation & Results

In [1]:
%matplotlib widget

In [2]:
import os

import torch
import torchio as tio

from obb import OBB
from itkwidgets import view
from ipywidgets import widgets

from experiment import SegmentationModel3d
import numpy as np
import matplotlib.pyplot as plt

from scipy.ndimage import label, generate_binary_structure
from torch.utils.data import DataLoader
from pathlib import Path

from scipy import ndimage
import nibabel as nib
from statistics import mean
import json

In [3]:
#load latest checkpoint from our model
path = "../checkpoints/confused-elevator-180/epoch=197-avg_train_jaccard=0.00.ckpt"

In [4]:
#set default hyperparameters
defaults = {
    'learning_rate': 0.0001,
    'loss': 'dice',
    'alpha': 0.9,
    'blocks': 4,
    'batch_size': 2,
    'initial_features': 64,

    'p_dropout': 0.0,

    'p_affine_or_elastic': 0.0,
    'p_elastic': 0.2,
    'p_affine': 0.8,

    'patch_size': 48,
    'samples_per_volume': 10,
    'queue_length': 80,
    'patch_overlap': 4,
    'random_sample_ratio': 4,

    'log_image_every_n': 3,

    'data_path': '/data/training',
}

In [5]:
#load model from checkpoint
checkpoint = torch.load(path)
model = SegmentationModel3d(defaults)
model.load_state_dict(checkpoint["state_dict"])
model = model.cuda()

## 1. Load data and pre-process

In [6]:
#load files
test_files = list(Path('/data/test').glob('*.*'))
files_orig = sorted(
    list(filter(lambda file: 'orig.nii.gz' in str(file), test_files)))

In [7]:
#create dataset and loader
preprocessing = tio.RescaleIntensity(out_min_max=(0, 1))
subjects = [
    tio.Subject(
        t1=tio.ScalarImage(orig)
    )
    for orig in files_orig]

test_set =  tio.SubjectsDataset(subjects, transform=preprocessing)   
test_loader = DataLoader(test_set, batch_size=1)

In [8]:
#transform dataset to TorchIO subject with affine matrix and path
def transform_to_subject(batch):
    batch['t1'] = tio.Image(tensor=batch['t1']['data'][0].to('cuda:0'), affine=batch['t1']['affine'][0], path=batch['t1']['path'])
    return tio.Subject(batch)

In [ ]:
#define batch prediction
def predict_on_batch(batch):
    subj = transform_to_subject(batch)
    with torch.no_grad():
        pred = model(subj)
    return (subj, pred)

## 2. Define Post-Processing
As the output of our model is ocassionally noisy, we employ multiple forms of post-processing on the outputs: thresholding, opening and sparseness analysis. Additionally we need to separate multiple aneurysms that might be occuring in one input.

In [9]:
def postprocess(result):
    '''
        Postprocessing the model output
        :param result: The raw model segmentation output for an entire volume
        :return: Individual bounding box and NIFTI mask per found aneurysm
    '''
    #load image, affine and id from subject
    subj = result[0]
    affine = subj['t1'].affine
    cid = subj['t1'].path[0].stem[0:4]
    #exception for one case
    if cid == 'A144':
        cid = subj['t1'].path[0].stem[0:6]
    
    #Set dummy for processing time
    ptime = -1
    
    #threshold prediction volume to be above a certain probability
    pred = result[1].numpy() > 0.99
    
    #perform morphological opening
    pred = ndimage.binary_opening(pred[0],iterations=3)
    #convert to binary array
    pred = pred.astype('int')

    #find connected components in prediction mask
    labeled_array, num_features = label(pred)
    #initialize empty list to store the detected aneurysms
    detected = []
        
    accepted_aneurysms = 0
    #loop through separate features in prediction
    for f in range(num_features):
        #get data points from aneurysm
        m = np.where(labeled_array == f+1,labeled_array,0)
        if np.count_nonzero(m) < 500: 
            #if aneurysm is very sparse it's probably noise, so we discard it
            continue

        #save nifti image of mask
        acc = str(accepted_aneurysms).zfill(2)
        ni_img = nib.Nifti1Image(m, affine)
        nib.save(ni_img, f"../results/{cid}_{acc}_output.nii.gz")
        accepted_aneurysms += 1

        #compute bounding box for aneurysm
        bb = OBB(m, affine)
        detected.append(bb.bounding_box())
    
    #if there is no detected aneurysm, store emtpy mask
    if accepted_aneurysms == 0:
        empty = np.zeros_like(labeled_array)
        ni_img = nib.Nifti1Image(empty, affine)
        nib.save(ni_img, f"../results/{cid}_00_output.nii.gz")
        

    
    return {
             "dataset_id":cid,
             "processing_time_in_seconds":ptime,
             "candidates":detected
          }

## 3. Run inference

In [16]:
final = []
for index, batch in enumerate(test_loader):
    output = predict_on_batch(batch)
    json_output = postprocess(output)
    final.append(json_output)
    print(f'finished run {index+1}/{len(test_loader)}')
    

finished run 1/22
finished run 2/22
finished run 3/22
finished run 4/22
finished run 5/22
finished run 6/22
finished run 7/22
finished run 8/22
finished run 9/22
finished run 10/22
finished run 11/22
finished run 12/22
finished run 13/22
finished run 14/22
finished run 15/22
finished run 16/22
finished run 17/22
finished run 18/22
finished run 19/22
finished run 20/22
finished run 21/22
finished run 22/22


In [18]:
#Write the final JSON output
o = {
       "username":"acorn",
       "task_1_results": final
    }

with open('../results/detection.json', 'w') as f:
    json.dump(o, f)